In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import os
import re

In [2]:
html = open('search_results.html')
soup = BeautifulSoup(html, 'html.parser')

In [3]:
# Extract all links inside result-item medicine divs
urls = [a['href'] for a in soup.select('.result-item.medicine a[href]')]

In [4]:
base_url ='https://www.farmacotherapeutischkompas.nl'

In [5]:
# Function to fetch and extract full 'recipe' sections containing ' FNA'
def extract_fna_recipe(url):
    full_url = base_url + url
    filename= url.split('/')[-1]
    filepath = f'pages/{filename}.html'       
    try:
        if not os.path.exists(filepath):
            try:
                response = requests.get(full_url)
                file = open(filepath, 'wb')
                file.write(response.content)
                file.close()

            except Exception as e:
                print(e)
            if response.status_code == 200:
                page_soup = BeautifulSoup(response.text, 'html.parser')
            
        else:
            try:
                file = open(filepath)
                page_soup = BeautifulSoup(file.read(), 'html.parser')
                file.close()
            except Exception as e:
                print(e)
            
        fna_recipes = [section for section in page_soup.find_all('section', class_='recipe') if 'FNA' in section.get_text()]

        atc = page_soup.find_all('span',class_='byline-item')
        atc_text = atc[-1].get_text() if atc else ""
            
        return fna_recipes, atc_text
    
    except Exception as e:
        print(e)
        return [],""

In [6]:
def get_ATC():
    pass

In [45]:
columns = ["Brandname", "ActiveIngredient", "Strength", "DosageForm", "ATC", "PrescribingProduct"]

#Initialize library
records = []

# Iterate through URLs and extract relevant content

for url in urls:
    print(base_url+url)
    fna_recipes, atc = extract_fna_recipe(url)

    #print(fna_recipes)
    for section in fna_recipes:
        name = section.find("span", class_="name").text.strip() if section.find("span", class_="name") else ""

        try:
            doses = [dose for dose in section.find_all(class_='doses')]
            #print(doses)

            for dose in doses:
                try:
                    dosage_form = dose.find("dd", title="Toedieningsvorm").text.strip() if dose.find("dd", title="Toedieningsvorm") else ""
                except Exception as e:
                    print(e)
                            
                try:
                    strengths_text =  dose.find("dd", title="Sterkte").text.strip()
                    print(repr(strengths_text))
                    #strenghts = re.split(r",\s*(?!\d,\d+\s*[a-zA-Z])", strengths_text)
                    
                    strengths = re.split(r",\s", strengths_text,flags=re.IGNORECASE)
                    print(strengths)

                except Exception as e:
                    print('doses - strengths not found: ',e)
                    strengths=[]
                
                if strengths:
                    for strength in strengths:
                        records.append({
                            "Brandname": "",
                            "ActiveIngredient": name,
                            "Strength": strength,
                            "DosageForm": dosage_form,
                            "ATC": atc,
                            "PrescribingProduct": name + " - " + strength + " - " + dosage_form
                        })
                else:
                    records.append({
                        "Brandname": "",
                        "ActiveIngredient": name,
                        "Strength": "",
                        "DosageForm": dosage_form,
                        "ATC": atc,
                        "PrescribingProduct": name + " - " + "" + " - " + dosage_form
                    })
        ####################################################
        except Exception as e:
            print(e)
        
            try:
                dosage_form = section.find("dd", title="Toedieningsvorm").text.strip() if section.find("dd", title="Toedieningsvorm") else ""
                #dosage_form = section.find_all("dd", title="Toedieningsvorm")

                print(dosage_form)
            except Exception as e:
                print(e)
                        
            try:
                strengths=  dose.find("dd", title="Sterkte").text.strip().split(',')
                #strenghts = re.findall(r'\d+(?:,\d+)?(?:\s*[a-zA-Z]+)?', strengths_text)
                print('no doses - strengths found: ',strengths)
            except Exception as e:
                print('no doses - strengths not found: ',e)
                strengths=[]
            
            if strengths:
                for strength in strengths:
                    records.append({
                        "Brandname": "",
                        "ActiveIngredient": name,
                        "Strength": strength,
                        "DosageForm": dosage_form,
                        "ATC": atc,
                        "PrescribingProduct": name + " - " + strength + " - " + dosage_form
                    })
            else:
                records.append({
                    "Brandname": "",
                    "ActiveIngredient": name,
                    "Strength": "",
                    "DosageForm": dosage_form,
                    "ATC": atc,
                    "PrescribingProduct": name + " - " + "" + " - " + dosage_form
                })

#Create dataframe
df = pd.DataFrame(records, columns=columns)
# Display DataFrame
#display(df)





https://www.farmacotherapeutischkompas.nl/bladeren/preparaatteksten/h/hydrocortison__systemisch_
'1 mg, 2 mg, 3 mg, 4 mg, 5 mg, 6 mg, 7 mg, 8 mg, 9 mg, 10 mg'
['1 mg', '2 mg', '3 mg', '4 mg', '5 mg', '6 mg', '7 mg', '8 mg', '9 mg', '10 mg']
https://www.farmacotherapeutischkompas.nl/bladeren/groepsteksten/corticosteroiden__systemisch
https://www.farmacotherapeutischkompas.nl/bladeren/preparaatteksten/c/colecalciferol
'50.000 IE/ml'
['50.000 IE/ml']
https://www.farmacotherapeutischkompas.nl/bladeren/groepsteksten/vitamine_d_en_analoga
https://www.farmacotherapeutischkompas.nl/bladeren/preparaatteksten/i/indifferente__vette__creme
doses - strengths not found:  'NoneType' object has no attribute 'text'
'10 %, 20 %, 50 %'
['10 %', '20 %', '50 %']
doses - strengths not found:  'NoneType' object has no attribute 'text'
'10 %, 20 %, 50 %'
['10 %', '20 %', '50 %']
'23 %'
['23 %']
'23 %'
['23 %']
https://www.farmacotherapeutischkompas.nl/bladeren/groepsteksten/indifferente_middelen
https://www.f

In [117]:
from datetime import date
output_filename = f'edups_FNA_formularium_v{date.today()}.csv'

df.to_csv(output_filename, index=False, encoding='latin-1',sep=';', decimal=',', errors='ignore')


In [40]:
string = '1 mg, 2 mg, 3 mg, 4 mg, 0,1 mg, 0,01 mg, 10%, 20%, 2 mg/g, 0,3 mg/g'
print(string)

print('regular split: ',string.split(','))


print('regex split by comma: ', re.split(',', string))

print('regex split complex: ', re.split(',\s', string))

1 mg, 2 mg, 3 mg, 4 mg, 0,1 mg, 0,01 mg, 10%, 20%, 2 mg/g, 0,3 mg/g
regular split:  ['1 mg', ' 2 mg', ' 3 mg', ' 4 mg', ' 0', '1 mg', ' 0', '01 mg', ' 10%', ' 20%', ' 2 mg/g', ' 0', '3 mg/g']
regex split by comma:  ['1 mg', ' 2 mg', ' 3 mg', ' 4 mg', ' 0', '1 mg', ' 0', '01 mg', ' 10%', ' 20%', ' 2 mg/g', ' 0', '3 mg/g']
regex split complex:  ['1 mg', '2 mg', '3 mg', '4 mg', '0,1 mg', '0,01 mg', '10%', '20%', '2 mg/g', '0,3 mg/g']
